In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn
import cv2
import random

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


import tensorflow as tf
import keras
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
data='/content/drive/MyDrive/Colab Notebooks/my_projects/flower_recog/data/'
folders=os.listdir("/content/drive/MyDrive/Colab Notebooks/my_projects/flower_recog/data/")
print(folders)

['Sunflower', 'lilly_of_the_valley', 'scilia', 'orris_root', 'toad_lilly', 'flowering_dogwood']


In [ ]:
size=64,64

In [ ]:
image_train=[]
image_labels=[]
image_names=[]

for folder in folders:
    for each in os.listdir(os.path.join(data,folder)):
        if each.endswith('jpg'):
            image_names.append(os.path.join(data,folder,each))
            image_labels.append(folder)
            img=cv2.imread(os.path.join(data,folder,each))
            img_in=cv2.resize(img,size)
            image_train.append(img_in)
        else:
            continue

In [ ]:
union_list=list(zip(image_train,image_labels))
random.shuffle(union_list)
train,labels=zip(*union_list)
X=np.array(train)
Y=np.array(labels)

In [ ]:
X=X/255.0

In [ ]:
le=LabelEncoder()
Y=le.fit_transform(Y)
Y=to_categorical(Y,num_classes=6)

In [ ]:
X_train,X_val,Y_train,Y_val=train_test_split(X,Y,test_size=0.1)

In [ ]:
from keras.applications.vgg16 import VGG16
vgg16_model=keras.applications.vgg16.VGG16(include_top=False,input_shape=(64,64,3))

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
model=Sequential()
for layers in vgg16_model.layers:
    model.add(layers)

In [ ]:
model.add(Flatten())
model.add(Dense(4096,activation='relu'))
model.add(Dense(4096,activation='relu'))
for layer in model.layers:
    layer.trainable=False
model.add(Dense(6,activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 16, 16, 256)       2

In [ ]:
model.compile(optimizer="adam",loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.1,
        height_shift_range=0.1,  
        horizontal_flip=False,
        vertical_flip=False) 

datagen.fit(X_train)

In [ ]:
history=model.fit_generator(datagen.flow(X_train,Y_train,batch_size=10),epochs=5,validation_data=(X_val,Y_val),
                          verbose=1,steps_per_epoch=X_train.shape[0]/10)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
43/43 [==============================] - 39s 549ms/step - loss: 1.7873 - accuracy: 0.1933 - val_loss: 1.6549 - val_accuracy: 0.4167
Epoch 2/5
43/43 [==============================] - 22s 520ms/step - loss: 1.6329 - accuracy: 0.3916 - val_loss: 1.5451 - val_accuracy: 0.4167
Epoch 3/5
43/43 [==============================] - 22s 517ms/step - loss: 1.5042 - accuracy: 0.5363 - val_loss: 1.4291 - val_accuracy: 0.5833
Epoch 4/5
43/43 [==============================] - 23s 521ms/step - loss: 1.3958 - accuracy: 0.6036 - val_loss: 1.3871 - val_accuracy: 0.5833
Epoch 5/5
43/43 [==============================] - 22s 519ms/step - loss: 1.3139 - accuracy: 0.6918 - val_loss: 1.3182 - val_accuracy: 0.5417
